In [ ]:
!pip install nltk
!pip install fuzzywuzzy



In [ ]:
import pandas as pd
import nltk
from fuzzywuzzy import fuzz

nltk.download('wordnet')


/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
df = pd.read_csv('train_data.csv')


In [ ]:
import pandas as pd
from nltk.corpus import wordnet

def get_synonyms(word):
    synonyms = []
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.append(lemma.name())
    return set(synonyms)

def get_similar_spellings(word):
    spellings = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            spelling_ratio = fuzz.ratio(word, lemma.name())
            if spelling_ratio > 60:
                spellings.add(lemma.name())
    return spellings

In [ ]:
warranty_synonyms = get_synonyms("warranty")
guarantee_synonyms = get_synonyms("guarantee")

warranty_similar_spellings = get_similar_spellings("warranty")
guarantee_similar_spellings = get_similar_spellings("guarantee")


all_terms = list(
    warranty_synonyms.union(warranty_similar_spellings) |
    guarantee_synonyms.union(guarantee_similar_spellings) |
    {'warranty', 'guarantee'}
)

print(all_terms)

['insure', 'guaranty', 'ensure', 'warrantee', 'undertake', 'warrant', 'guarantee', 'vouch', 'secure', 'warranty', 'assure']


In [ ]:
filtered_rows = df[df['reviewText'].str.contains('|'.join(all_terms), case=False, na=False)]
filtered_rows.to_csv('filtered_data.csv', index=False)



In [ ]:
mean_overall = filtered_rows.groupby('asin')['overall'].mean()
print(mean_overall)


asin
0972683275    5.0
1616825375    5.0
9806010728    4.0
B000001OM4    4.0
B000001OM5    5.0
             ... 
B00009R89L    4.5
B00009R8T5    2.0
B00009R8XD    5.0
B00009R9BF    5.0
B00009RDIF    5.0
Name: overall, Length: 153, dtype: float64


In [ ]:
mean_overall.to_csv('mean_overall_ratings.csv', header=True)
